In [1]:
# Importações

import tensorflow
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import PIL
from sklearn import preprocessing, model_selection
import numpy as np

Adicionando as configurações de diretório e outros artefatos para o modelo

In [2]:
# diretorio da localização das imagens de treino
DIR = r"C:\Users\perei\Desktop\universidade\inteligencia artificial\utfpr_ia"
CATEGORIAS = ['mask', 'no_mask']

# iniciando os arrays de Data e labels vazios
data = []
labels = []

Adicionando as imagens de treino

In [3]:
for categoria in CATEGORIAS:
    path = os.path.join(DIR, categoria)
    for imagem in os.listdir(path):
        image_path = os.path.join(path, imagem)
        image = keras.preprocessing.image.load_img(image_path, target_size=(224,224))
        image = keras.preprocessing.image.img_to_array(image)
        image = keras.applications.mobilenet_v2.preprocess_input(image)

        data.append(image)
        labels.append(categoria)

C:\Users\perei\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Transformando valores categóricos em vetores binários esparso

In [4]:
bin_labels = preprocessing.LabelBinarizer()
labels = bin_labels.fit_transform(labels)
labels = keras.utils.to_categorical(labels)

Convertendo os arrays de data e labels para o formato float32 com o numpy

In [5]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

Splitando o array em subset de testes e trainamento

In [6]:
(trainX, testX, trainY, testY) = model_selection.train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

Gerador de imagem de treinamento

In [7]:
aumento = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest"
)

Carregamento da rede MobileNetV2

In [8]:
modeloBase = keras.applications.MobileNetV2(
    weights="imagenet", include_top=False, input_tensor=keras.layers.Input(shape=(244, 244, 3))
)

cabecalhoModelo = modeloBase.output
cabecalhoModelo = keras.layers.AveragePooling2D(pool_size=(7,7))(cabecalhoModelo) # pesquisar sobre o AvaregePooling
cabecalhoModelo = keras.layers.Flatten(name="flatten")(cabecalhoModelo)
cabecalhoModelo = keras.layers.Dense(128, activation="relu")(cabecalhoModelo)
cabecalhoModelo = keras.layers.Dropout(0.5)(cabecalhoModelo)
cabecalhoModelo = keras.layers.Dense(2, activation="softmax")(cabecalhoModelo)

Criando o modelo de treinamento

In [9]:
modelo = keras.Model(inputs=modeloBase.input, outputs=cabecalhoModelo)

Fazendo um loop em cima das camadas do modelo base

In [10]:
for camada in modeloBase.layers:
    camada.trainable = False

Compilando o nosso modelo

In [ ]:
# definindo o otimizador
otimizador = keras.optimizers.Adam(lr=1e-4, decay=1e-4/20)

modelo.compile(loss="binary_crossentropy", optimizer=otimizador, metrics=['accuracy'])

Treinando o cabeçalho do modelo